In [164]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [165]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1HOUR,
                                      str(365) + ' days ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [166]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-07-26 11:00:00,29192.54,29218.00,29160.33,29175.39,792.42820
2023-07-26 12:00:00,29175.39,29243.43,29166.36,29229.13,711.11796
2023-07-26 13:00:00,29229.13,29321.69,29229.12,29306.54,1502.97312
2023-07-26 14:00:00,29306.54,29354.00,29206.89,29267.54,1814.07053
2023-07-26 15:00:00,29267.54,29337.33,29267.12,29324.00,1152.09303
...,...,...,...,...,...
2024-07-25 06:00:00,64263.01,64369.05,64086.43,64324.01,647.68953
2024-07-25 07:00:00,64324.00,64390.00,64071.51,64134.00,1004.28015
2024-07-25 08:00:00,64134.00,64419.71,63878.00,64352.01,1572.60803


In [167]:
df['SMA50'] = df['Close'].rolling(50).mean()
df['SMA200'] = df['Close'].rolling(200).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd(df['Close'])
df['MACD_sig'] = trend.macd_signal(df['Close'])
df['MACD_diff'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [168]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2023-08-03 18:00:00,29230.24,29271.03,29137.55,29269.76,1426.65745,29314.8130,29287.67195,52.677139,2.510112,-18.958942,21.469053,-0.006660,21.946972,20.257611,20.739235
2023-08-03 19:00:00,29269.76,29300.00,29220.00,29281.32,603.90476,29321.3592,29288.20160,53.323962,6.670487,-13.833056,20.503542,-0.004444,20.751988,19.324545,21.451934
2023-08-03 20:00:00,29281.32,29361.14,29271.99,29304.00,1119.68516,29321.9992,29288.57595,54.634092,11.663253,-8.733794,20.397047,0.000986,20.218236,18.312306,23.920605
2023-08-03 21:00:00,29304.00,29338.00,29230.73,29253.91,845.80868,29321.5892,29288.31280,51.215006,11.446267,-4.697782,16.144049,-0.002302,19.251535,19.593160,22.400173
2023-08-03 22:00:00,29253.91,29269.31,29165.46,29235.25,1071.03801,29321.6994,29288.15135,49.960653,9.657274,-1.826771,11.484045,0.007212,18.086569,22.281589,21.008003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25 06:00:00,64263.01,64369.05,64086.43,64324.01,647.68953,65922.4838,66005.44845,33.671187,-540.206341,-392.515298,-147.691042,-0.022204,32.918806,30.752622,9.202379
2024-07-25 07:00:00,64324.00,64390.00,64071.51,64134.00,1004.28015,65876.6038,66000.89850,31.827164,-555.539049,-425.120049,-130.419001,-0.023330,34.331480,29.306975,9.079006
2024-07-25 08:00:00,64134.00,64419.71,63878.00,64352.01,1572.60803,65833.2572,65995.13150,36.147994,-543.829815,-448.862002,-94.967813,-0.023166,35.893912,29.815563,8.359222


In [169]:
#SMA Golden Cross & Death Cross
sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

In [170]:
# MACD Below 0 Cross
macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

In [171]:
# RSI Overbought & Oversold
rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

In [172]:
# ADX Trend Confirmation
adx_buy_cond = (df['ADX'] > 25) & (df['+DI'] > df['-DI'])
adx_sell_cond = (df['ADX'] > 25) & (df['-DI'] > df['+DI'])
df['Signal_adx'] = np.where(adx_buy_cond, 1, np.where(adx_sell_cond, -1, 0))

In [173]:
# Momentum Trade
momentum_buy_cond = (df['Momentum'] > 0) & (df['Momentum'].shift(1) > 0) & (df['Momentum'].shift(2) > 0) & (df['Momentum'].shift(3) > 0) & (df['Momentum'].shift(4) > 0) \
                    & (df['Momentum'].shift(5) > 0) & (df['Momentum'].shift(6) > 0) & (df['Momentum'].shift(7) > 0) & (df['Momentum'].shift(8) > 0) & (df['Momentum'].shift(9) > 0)
momentum_sell_cond = (df['Momentum'] < 0) & (df['Momentum'].shift(1) < 0) & (df['Momentum'].shift(2) < 0) & (df['Momentum'].shift(3) < 0) & (df['Momentum'].shift(4) < 0) \
                    & (df['Momentum'].shift(5) < 0) & (df['Momentum'].shift(6) < 0) & (df['Momentum'].shift(7) < 0) & (df['Momentum'].shift(8) < 0) & (df['Momentum'].shift(9) < 0)
df['Signal_momentum'] = np.where(momentum_buy_cond, 1, np.where(momentum_sell_cond, -1, 0))

In [174]:
df['Buy/Sell Consensus'] = df['Signal_sma'] + df['Signal_macd'] + df['Signal_rsi'] + df['Signal_adx'] + df['Signal_momentum']


# Run Test Trades

In [175]:
historical_df = df[df['Buy/Sell Consensus'] != 0]
historical_df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-04 03:00:00,29166.00,29178.70,29136.17,29161.29,540.30101,29283.3048,29284.53765,45.065013,-8.882457,-2.177342,...,-0.000626,16.218884,25.273953,16.865075,-1,0,0,0,0,-1
2023-08-04 04:00:00,29161.28,29209.21,29134.70,29205.98,420.45225,29271.2072,29282.67710,48.836262,-8.673082,-3.476490,...,0.000685,16.044748,23.937250,18.138761,-1,0,0,0,0,-1
2023-08-04 13:00:00,29222.54,29333.08,29208.27,29240.72,1520.05490,29204.9796,29272.61125,52.961854,-5.608659,-7.841526,...,0.006767,12.199406,17.965713,26.676983,0,1,0,0,0,1
2023-08-04 17:00:00,29265.39,29269.24,29197.20,29200.01,758.35684,29195.4238,29268.54015,47.698853,4.058527,-0.914881,...,0.001853,11.318841,21.486484,21.886665,0,0,0,0,1,1
2023-08-05 03:00:00,29086.04,29102.45,29082.25,29094.53,430.05398,29177.0162,29255.00915,44.974886,-35.581369,-32.776878,...,-0.005448,25.784218,32.488874,14.889020,0,0,0,-1,-1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25 06:00:00,64263.01,64369.05,64086.43,64324.01,647.68953,65922.4838,66005.44845,33.671187,-540.206341,-392.515298,...,-0.022204,32.918806,30.752622,9.202379,0,0,0,-1,-1,-2
2024-07-25 07:00:00,64324.00,64390.00,64071.51,64134.00,1004.28015,65876.6038,66000.89850,31.827164,-555.539049,-425.120049,...,-0.023330,34.331480,29.306975,9.079006,0,0,0,-1,-1,-2
2024-07-25 08:00:00,64134.00,64419.71,63878.00,64352.01,1572.60803,65833.2572,65995.13150,36.147994,-543.829815,-448.862002,...,-0.023166,35.893912,29.815563,8.359222,0,0,0,-1,-1,-2


In [176]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0

In [177]:
for idx, row in historical_df.iterrows():
    if (row['Buy/Sell Consensus'] >= 2) & (open_trade == False):
        coin = (balance / row['Close']) *0.999
        bought_price = row['Close']
        open_trade = True
    elif (row['Buy/Sell Consensus'] <= -2) & (open_trade == True):
        open_trade = False
        print(f"PnL:{(row['Close']-bought_price)/bought_price * 100}")
        balance = coin * row['Close'] *0.999
        print(balance)

PnL:-2.064272812910123
977.3995366844288
PnL:-0.5811474274025049
969.7769373321044
PnL:-0.43594123937550555
963.6191467221361
PnL:-0.961584349247738
952.4453839023953
PnL:-1.8281851075985587
933.1637884303292
PnL:0.8161880670702112
938.8995403780423
PnL:3.6446758906559706
971.1741199119388
PnL:-0.7801320206728518
961.6714478644456
PnL:26.66736781001343
1215.688880294268
PnL:-0.37111703639237975
1208.75610842372
PnL:4.501794165506847
1260.6467399189912
PnL:-2.8569409914083215
1222.1827694673004
PnL:-2.4712501615309947
1189.5968086306063
PnL:-0.06560376239052165
1186.4399444065216
PnL:14.67976366243981
1357.8866718002212
PnL:-4.62065713265652
1292.554392820784
PnL:-1.4579873160448351
1271.1629692021493
PnL:0.4515956924288818
1274.3509563454738
PnL:-2.029125346527538
1245.9970410231567
PnL:-1.6350823410144273
1223.1739411329165
PnL:-2.805338941471094
1186.4832355676735
PnL:2.5357173872174137
1214.1371756429432
PnL:-2.7780660545973204
1178.0480080319826
PnL:-1.3106624322889444
1160.2837224

In [178]:
1 - 1/1000

0.999

In [179]:
balance

1501.8301514026277